In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
import json
import time

# --- ⚙️ 設定：本地資料庫檔案名稱 ---
# 請確保這個檔案跟程式在同一個資料夾內
LOCAL_DB_FILENAME = "my_prompts.json" 

# --- 🗣️ 介面語言包 (UI Localization) ---
# 保留原本的雙語介面設定
UI_TEXT = {
    "zh": {
        "guide": """
        <div style="background:#f8fafc; padding:8px 12px; border-radius:6px; font-size:13px; color:#475569; margin-bottom:15px; border-left:3px solid #3b82f6; display:flex; align-items:center;">
            <span style="margin-right:8px;">💡</span>
            <span>👈 <b>左側自由探索</b>模組與參數，設定完成後別忘了點擊右側 <b>[複製 Prompt]</b> 帶走你的咒語！</span>
        </div>
        """,
        "lbl_lib": "📚 資料庫來源:",
        "lbl_role": "📂 角色分類:",
        "lbl_task": "⚡ 具體任務:",
        "lbl_config": "CONFIG (設定)",
        "lbl_vars": "VARIABLES (參數)",
        "lbl_result": "RESULT (結果)",
        "lbl_output_mode": "🎛️ 輸出模式:",
        "btn_copy": "複製 Prompt (Copy)",
        "btn_copied": "已複製! ✅",
        "status_online": "🟢 本地模式 (Offline)",
        "mode_options": [
            ("🚀 智能預設 (Auto)", ""),
            ("🤫 靜默接收模式 (Silent Ack)", "【系統提示】：請接收以下輸入內容，但【先不要執行】任何任務。你只需要簡單回覆一句『🆗 收到，我已理解您的指令，請輸入 OK 讓我開始執行。』，然後等待使用者的下一個信號。除此之外不要輸出任何其他內容。\n\n----------------\n\n"),
            ("🎨 繪圖咒語大師 (Midjourney)", "【系統提示】：請扮演專業的 AI 繪圖詠唱師 (Prompt Engineer)。請根據用戶下方的描述，撰寫給 Midjourney v6 使用的「英文提示詞」。包含：主體、藝術風格、光影、鏡頭角度及長寬比 (--ar)。不需要解釋，直接輸出提示詞即可。\n\n----------------\n\n"),
            ("🎬 影片運鏡導演 (Sora/Runway)", "【系統提示】：請扮演專業的 AI 影片導演。請根據用戶需求，撰寫詳細的影片生成提示詞。重點描述：運鏡方式 (Pan, Zoom, Dolly)、光影氛圍、物理動態以及場景連貫性。格式請針對 Runway Gen-2 或 Sora 優化。\n\n----------------\n\n"),
            ("📝 純文字輸出 (Text Only)", "【系統提示】：系統強制覆寫：輸出結果必須嚴格限制為「純文字」。除非用戶特別要求，否則【禁止】生成圖片、程式碼區塊或呼叫任何多媒體工具。請保持版面乾淨易讀。\n\n----------------\n\n"),
            ("🐍 純代碼模式 (Code Only)", "【系統提示】：你現在是一台無情的寫程式機器。針對用戶的問題，【只輸出程式碼區塊】(Python/JS/HTML 等)。不要有任何開場白、結尾、解釋或註解。給我 code 就好。\n\n----------------\n\n"),
            ("🐞 除錯醫生 (Bug Fixer)", "【系統提示】：請分析以下程式碼的錯誤 (Bug)。先用簡短的一句話解釋錯誤原因，然後提供【修正後的完整程式碼區塊】。請注重程式的安全性和執行效率。\n\n----------------\n\n"),
            ("👶 費曼學習法 (ELI5)", "【系統提示】：請把以下的內容解釋給我聽，假設我是一個只有 5 歲的小朋友。使用簡單的生活譬喻，避免專業術語，語氣要生動有趣。\n\n----------------\n\n"),
            ("🧠 深度思考鏈 (CoT)", "【系統提示】：請不要直接給我答案。請使用「思維鏈 (Chain of Thought)」模式，一步一步地思考。將問題拆解，分析利弊，展示你的推論過程，最後再給出結論。\n\n----------------\n\n"),
            ("📹 YouTube 腳本 (Script)", "【系統提示】：請根據輸入的主題，撰寫一份高流量的 YouTube 影片腳本。結構需包含：黃金 5 秒開場 (Hook)、前言、三個核心重點段落、以及結尾行動呼籲 (CTA)。語氣要像 YouTuber 一樣口語且有吸引力。\n\n----------------\n\n"),
            ("📧 Email 潤飾 (Polish)", "【系統提示】：請將以下草稿潤飾為一封「專業、禮貌且簡潔」的商務電子郵件。修正語法錯誤，並提升整體的專業語氣。\n\n----------------\n\n"),
            ("📋 重點摘要 (TL;DR)", "【系統提示】：請閱讀以下內容，並以列點方式 (Bullet points) 提供「重點摘要 (TL;DR)」。只保留最關鍵的資訊，過濾掉廢話。\n\n----------------\n\n"),
            ("😈 魔鬼代言人 (Critic)", "【系統提示】：請擔任「魔鬼代言人」的角色。針對以下的觀點或計畫，找出邏輯漏洞、潛在風險或缺點。請毫不留情地進行批判性審查，但要保持建設性。\n\n----------------\n\n"),
            ("📊 數據結構化 (JSON)", "【系統提示】：請將以下資訊轉換為標準的 JSON 格式。不要使用 Markdown 或任何對話文字，直接輸出 Raw JSON 數據。\n\n----------------\n\n"),
            ("🌍 翻譯蒟蒻 (Translate)", "【系統提示】：請將以下內容翻譯成自然、通順的「台灣繁體中文」。避免使用機器翻譯的生硬語氣，請使用台灣在地的習慣用語。\n\n----------------\n\n"),
            ("💡 創意風暴 (Brainstorming)", "【系統提示】：請針對用戶提供的主題，進行腦力激盪。列出 10 個獨特、創新且視角多元的點子。請跳脫框架思考 (Think outside the box)。\n\n----------------\n\n"),
            ("🧼 洗稿大師 (Rewrite)", "【系統提示】：請改寫以下文章。保留原本的核心語意，但大幅改變句型結構和用詞，使其成為一篇無法被檢測出重複的原創文章。\n\n----------------\n\n"),
            ("🔮 玄學大師 (Mystic)", "【系統提示】：請用玄學、命理或塔羅占卜師的口吻回答以下問題。多使用與星象、命運、能量流動相關的隱喻，讓答案充滿神秘感。\n\n----------------\n\n"),
            ("🛌 懶人包教學 (Step-by-Step)", "【系統提示】：請將這項任務拆解為一份簡單易懂的「傻瓜教學」。請使用編號步驟 (Step 1, Step 2...)，並確保每個步驟都具體可行。\n\n----------------\n\n")
        ]
    },
    "en": {
        "guide": """
        <div style="background:#f8fafc; padding:8px 12px; border-radius:6px; font-size:13px; color:#475569; margin-bottom:15px; border-left:3px solid #10b981; display:flex; align-items:center;">
            <span style="margin-right:8px;">💡</span>
            <span>👈 <b>Explore</b> the settings on the left, and don't forget to click <b>[Copy Prompt]</b> on the right!</span>
        </div>
        """,
        "lbl_lib": "📚 Local Library:",
        "lbl_role": "📂 Category:",
        "lbl_task": "⚡ Task:",
        "lbl_config": "CONFIG",
        "lbl_vars": "VARIABLES",
        "lbl_result": "RESULT",
        "lbl_output_mode": "🎛️ Output Mode:",
        "btn_copy": "Copy Prompt",
        "btn_copied": "Copied! ✅",
        "status_online": "🟢 Local Mode",
        "mode_options": [
            ("🚀 Auto / Default", ""),
            ("🤫 Silent Receiver (Ack Only)", "SYSTEM OVERRIDE: Receive the following input BUT DO NOT EXECUTE IT YET. Simply reply with '🆗 Received. Waiting for your command.' and wait for the user's next signal. Do not output anything else.\n\n----------------\n\n"),
            ("📝 Text Only Mode", "SYSTEM OVERRIDE: Output must be strictly text only. DO NOT generate images, code blocks, or multimedia tools unless requested.\n\n----------------\n\n"),
            ("🐍 Code Generator", "SYSTEM OVERRIDE: You are a coding machine. Output ONLY the code block to solve the problem. Do not provide explanations or comments. Just the code.\n\n----------------\n\n"),
            ("👶 ELI5 (Simple Logic)", "SYSTEM OVERRIDE: Explain the concept as if I am a 12-year-old beginner. Use simple analogies and avoid jargon.\n\n----------------\n\n"),
            ("🧠 Chain of Thought", "SYSTEM OVERRIDE: Think step-by-step. Break down the problem, analyze pros and cons, and show your reasoning process.\n\n----------------\n\n"),
            ("📧 Email Polisher", "SYSTEM OVERRIDE: Rewrite the draft into a professional, concise business email. Fix grammar and tone.\n\n----------------\n\n"),
            ("📋 TL;DR Summary", "SYSTEM OVERRIDE: Read the content and provide a bulleted summary of the key takeaways.\n\n----------------\n\n"),
            ("😈 Devil's Advocate", "SYSTEM OVERRIDE: Act as a critical reviewer. Find flaws, logic gaps, or potential risks in the idea. Be harsh but constructive.\n\n----------------\n\n"),
            ("🌍 Translate to Chinese (TW)", "SYSTEM OVERRIDE: Translate the following text into natural, fluent Traditional Chinese.\n\n----------------\n\n")
        ]
    }
}

class CloudCopilot:
    def __init__(self):
        self.lang = "zh"
        
        # 🔥 設定：指向本地檔案
        self.libraries = {
            "📂 離線全功能資料庫 (All-in-One)": LOCAL_DB_FILENAME
        }
        
        self.current_db = {}
        self.roles_data = {} 
        self.input_widgets = {}
        self.setup_ui()
        
        # 初始化載入
        self.load_local_index()

    def setup_ui(self):
        style = """
        <style>
            .app-wrapper { font-family: 'Segoe UI', sans-serif; max-width: 1000px; margin: 0 auto; background: #fff; border-radius: 12px; border: 1px solid #e5e7eb; box-shadow: 0 10px 15px -3px rgba(0, 0, 0, 0.1); overflow: hidden; }
            .header-bar { background: linear-gradient(135deg, #0f172a 0%, #334155 100%); color: white; padding: 16px 24px; display: flex; justify-content: space-between; align-items: center; }
            .section-label { font-size: 11px; font-weight: 800; color: #64748b; letter-spacing: 1px; margin-bottom: 8px; text-transform: uppercase; margin-top: 15px; }
            .status-badge { font-size: 12px; background: rgba(255,255,255,0.1); padding: 4px 12px; border-radius: 20px; backdrop-filter: blur(4px); }
        </style>
        """

        header_content = f"""
        <div class='app-wrapper'>
            <div class='header-bar'>
                <div style='display:flex; flex-direction:column;'>
                    <div style='font-size:20px; font-weight:600; display:flex; align-items:center; gap:8px;'>
                        ✈️ Prompt Copilot
                        <span style='font-size:12px; background:#3b82f6; padding:2px 8px; border-radius:4px; opacity:0.9;'>v7.3 Offline</span>
                    </div>
                    <div style='font-size:11px; opacity:0.8; margin-top:4px;'>Local Version (By Neurora_AITec)</div>
                </div>
                <div style='display:flex; align-items:center; gap:12px;'>
                    <div id='status' class='status-badge'>{UI_TEXT[self.lang]['status_online']}</div>
                </div>
            </div>
        </div>
        """
        self.header_html = widgets.HTML(style + header_content)

        self.lang_toggle = widgets.ToggleButtons(
            options=[('🇹🇼 Taiwan (繁體中文)', 'zh'), ('🌏 Global (English)', 'en')],
            value='zh',
            style={'button_width': '180px'},
            layout=widgets.Layout(margin='0 0 15px 0')
        )
        self.lang_toggle.observe(self.on_language_change, names='value')

        self.guide_html = widgets.HTML(UI_TEXT['zh']['guide'])

        # Left Panel
        self.lbl_config = widgets.HTML(f"<div class='section-label'>{UI_TEXT['zh']['lbl_config']}</div>")
        self.lbl_vars = widgets.HTML(f"<div class='section-label'>{UI_TEXT['zh']['lbl_vars']}</div>")

        self.lib_dd = widgets.Dropdown(description=UI_TEXT['zh']['lbl_lib'], layout=widgets.Layout(width='99%'), style={'description_width': 'initial'})
        self.role_dd = widgets.Dropdown(description=UI_TEXT['zh']['lbl_role'], layout=widgets.Layout(width='99%'))
        self.task_dd = widgets.Dropdown(description=UI_TEXT['zh']['lbl_task'], layout=widgets.Layout(width='99%'))
        self.inputs_box = widgets.VBox(layout=widgets.Layout(padding='10px 0'))

        # Right Panel
        self.lbl_result = widgets.HTML(f"<div class='section-label'>{UI_TEXT['zh']['lbl_result']}</div>")
        self.output_mode_dd = widgets.Dropdown(
            options=UI_TEXT['zh']['mode_options'],
            description=UI_TEXT['zh']['lbl_output_mode'],
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='100%', margin='0 0 10px 0')
        )
        self.result_area = widgets.Textarea(layout=widgets.Layout(width='100%', height='450px'))
        self.copy_btn = widgets.Button(description=UI_TEXT['zh']['btn_copy'], icon='copy', button_style='primary', layout=widgets.Layout(width='100%', margin='10px 0 0 0'))

        # Bind Events
        self.lib_dd.observe(self.on_library_change, names='value')
        self.role_dd.observe(self.on_role_change, names='value')
        self.task_dd.observe(self.on_task_change, names='value')
        self.output_mode_dd.observe(self.update_result, names='value')
        self.copy_btn.on_click(self.on_copy_click)

    def load_local_index(self):
        # 設定下拉選單，並自動載入第一個
        self.lib_dd.options = list(self.libraries.keys())
        if self.libraries:
            self.on_library_change({'new': list(self.libraries.keys())[0]})

    def on_language_change(self, change):
        self.lang = change['new']
        t = UI_TEXT[self.lang]

        self.guide_html.value = t['guide']
        self.lbl_config.value = f"<div class='section-label'>{t['lbl_config']}</div>"
        self.lbl_vars.value = f"<div class='section-label'>{t['lbl_vars']}</div>"
        self.lbl_result.value = f"<div class='section-label'>{t['lbl_result']}</div>"

        self.lib_dd.description = t['lbl_lib']
        self.role_dd.description = t['lbl_role']
        self.task_dd.description = t['lbl_task']
        self.output_mode_dd.description = t['lbl_output_mode']
        self.copy_btn.description = t['btn_copy']
        self.output_mode_dd.options = t['mode_options']

    def on_library_change(self, change):
        if not change['new'] or change['new'] not in self.libraries: return
        
        filename = self.libraries[change['new']]
        
        # 🔥 核心修改：從本地檔案讀取
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                self.current_db = json.load(f)
            self.refresh_roles_and_force_render()
        except FileNotFoundError:
            self.result_area.value = f"❌ 找不到檔案：{filename}\n請確認該檔案是否存在於同一個資料夾中。"
        except Exception as e:
            self.result_area.value = f"❌ 讀取錯誤: {e}"

    def refresh_roles_and_force_render(self):
        # 處理 JSON 結構 (相容有 roles 包裹或直接字典)
        if 'roles' in self.current_db:
            self.roles_data = self.current_db['roles']
        else:
            self.roles_data = self.current_db

        roles = list(self.roles_data.keys())
        self.role_dd.options = roles
        if roles:
            self.role_dd.value = roles[0]
            self.on_role_change({'new': roles[0]})
        else:
            self.role_dd.options = []
            self.task_dd.options = []
            self.inputs_box.children = []

    def on_role_change(self, change):
        role = change['new']
        if role and role in self.roles_data:
            tasks = list(self.roles_data[role].keys())
            self.task_dd.options = tasks
            if tasks:
                self.task_dd.value = tasks[0]
                self.on_task_change({'new': tasks[0]})

    def on_task_change(self, change):
        role, task = self.role_dd.value, self.task_dd.value
        if role and task and role in self.roles_data and task in self.roles_data[role]:
            self.render_inputs(self.roles_data[role][task])

    def render_inputs(self, task_data):
        self.input_widgets = {}
        ui = []
        if "description" in task_data:
            ui.append(widgets.HTML(f"<div style='color:#64748b; font-size:13px; margin-bottom:12px;'>ℹ️ {task_data['description']}</div>"))

        # 處理 vars 參數
        vars_data = task_data.get('vars', {})
        for k, v in vars_data.items():
            if k.endswith('__multi') and isinstance(v, list):
                clean_k = k.replace('__multi', '')
                ui.append(widgets.HTML(f"<div class='section-label'>{clean_k}</div>"))
                w = widgets.SelectMultiple(options=v, value=[v[0]], description='', rows=5)
                w.layout = widgets.Layout(width='99%')
            elif isinstance(v, list):
                w = widgets.Dropdown(options=v, value=v[0], description=k)
                w.layout = widgets.Layout(width='99%')
                w.style = {'description_width': 'initial'}
            else:
                w = widgets.Text(value=str(v), description=k)
                w.layout = widgets.Layout(width='99%')

            w.observe(self.update_result, names='value')
            self.input_widgets[k] = w
            if not k.endswith('__multi'):
                ui.append(widgets.VBox([w], layout=widgets.Layout(margin='0 0 10px 0')))
            else:
                ui.append(widgets.VBox([w], layout=widgets.Layout(margin='0 0 10px 0')))

        self.inputs_box.children = ui
        self.update_result(None)

    def update_result(self, change):
        try:
            role, task = self.role_dd.value, self.task_dd.value
            if not role or not task: return

            tmpl = self.roles_data[role][task]['template']
            vals = {}
            for k, w in self.input_widgets.items():
                if isinstance(w.value, tuple):
                    vals[k] = "、".join(w.value)
                else:
                    vals[k] = w.value

            raw_prompt = tmpl.format(**vals)
            prefix = self.output_mode_dd.value
            self.result_area.value = prefix + raw_prompt
        except Exception as e: 
            # 容錯處理：如果 template key 對不上，避免程式崩潰
            pass

    def on_copy_click(self, b):
        safe_content = json.dumps(self.result_area.value)
        display(Javascript(f"navigator.clipboard.writeText({safe_content})"))
        original_text = b.description
        b.description = UI_TEXT[self.lang]['btn_copied']
        time.sleep(1)
        b.description = original_text

    def display(self):
        left = widgets.VBox([
            self.lang_toggle,
            self.guide_html,
            self.lbl_config,
            self.lib_dd,
            self.role_dd,
            self.task_dd,
            widgets.HTML("<hr style='border-top:1px dashed #cbd5e1; margin:15px 0;'>"),
            self.lbl_vars,
            self.inputs_box
        ], layout=widgets.Layout(width='45%', padding='20px'))

        right = widgets.VBox([
            self.lbl_result,
            self.output_mode_dd,
            self.result_area,
            self.copy_btn
        ], layout=widgets.Layout(width='55%', padding='20px', background_color='#f8fafc'))

        display(self.header_html, widgets.VBox([widgets.HBox([left, right])], layout=widgets.Layout(border='1px solid #ddd', border_top='none', border_radius='0 0 12px 12px')))

# 啟動程式
clear_output()
app = CloudCopilot()
app.display()